In [3]:
import sys
import numpy

In [4]:
# enumerate binary trees of size n
# the standard form equations are B = U + V, U = z, V = B * B
# intermediate values are tabulated by a dict k -> (b_k, u_k, v_k)

idxB = 0
idxU = 1
idxV = 2

def tabulate(k, table):
    v_k = 0
    if k != 0:
        for i in range(1,k):
            v_k += table[i][idxB] * table[k-i][idxB]
    u_k = 1 if k == 1 else 0
    b_k = u_k + v_k
    table[k] = (b_k, u_k, v_k)
    return table
    
def BinaryTree(n):
    t = {}
    for k in range(n+1):
        t = tabulate(k, t)
    print t[n][idxB]
    return

for k in range(0,20):
    BinaryTree(k)

0
1
1
2
5
14
42
132
429
1430
4862
16796
58786
208012
742900
2674440
9694845
35357670
129644790
477638700


In [48]:
# enumerate combinatorial structures from standard form
# supports union, cartesian product and terminal rules
# intermediate values are tabulated by a dict k -> (dict r -> r_k)

class Union:
    def __init__(self, a, b, c): # a = b + c
        self.Value = a
        self.SubRule1 = b
        self.SubRule2 = c

class Product:
    def __init__(self, a, b, c): # a = b * c
        self.Value = a
        self.SubRule1 = b
        self.SubRule2 = c

class Terminal:
    def __init__(self, a, n): # a = z where |z| = n
        self.Value = a
        self.Magnitude = n

# Here's a good question: is it true that for any rule U, if k = 0 then u_k = 0?
# This is an assumption we are currently making to evaluate products

def evaluateRule(rule, k, table):
    if k not in table:
        table[k] = {}
    if k == 0:
        table[k][rule.Value] = 0
    elif isinstance(rule, Union):
        table[k][rule.Value] = table[k][rule.SubRule1] + table[k][rule.SubRule2]
    elif isinstance(rule, Product):
        table[k][rule.Value] = 0
        for i in range(1,k):
            table[k][rule.Value] += table[i][rule.SubRule1] * table[k-i][rule.SubRule2]
    elif isinstance(rule, Terminal):
        table[k][rule.Value] = 1 if rule.Magnitude == k else 0
    else: raise Exception('Unsupported rule')
    return table

# compute the valuations of a list of rules
# implementation of algorithm on page 28, section 1.4
def computeRuleValuations(rules):
    val = {}
    for r in rules:
        if isinstance(r, Terminal):
            val[r.Value] = r.Magnitude
        else:
            val[r.Value] = sys.maxint
    return valuate(val, rules)

def sortRulesByValuation(rules):
    d = computeRuleValuations(rules).items()
    d_sorted = sorted(d, key=lambda x:x[1], reverse=True)
    r_sorted = []
    for (k,v) in d_sorted:
        for r in rules:
            if r.Value == k:
                if isinstance(r, Terminal):
                    r_sorted = [r] + r_sorted
                else:
                    r_sorted.append(r)
    return r_sorted
    
# helper for computeRuleValuations()
def valuate(v, rules):
    done = True
    for r in rules:
        prev = v[r.Value]
        if isinstance(r, Terminal):
            continue
        elif isinstance(r, Union):
            v[r.Value] = min(v[r.SubRule1], v[r.SubRule2])
        elif isinstance(r, Product):
            v[r.Value] = v[r.SubRule1] + v[r.SubRule2]
        else: raise Exception('Unsupported rule')
        if v[r.Value] != prev: done = False
    if done: return v
    else: return valuate(v, rules)

# rules should be sorted in order of increasing minimality
def EnumerateFromStandardForm(rules, n):
    t = {}
    for k in range(n+1):
        for r in rules:
            t = evaluateRule(r, k, t)
    return t

# test on a binary tree
def BTree(n):
    rules = [Union('B', 'U', 'V'), Product('V', 'B', 'B'), Terminal('U', 1)]
    tab = EnumerateFromStandardForm(sortRulesByValuation(rules), n)
    print tab[n]['B']

# runable code goes here
for i in range(20):
    BTree(i)

0
1
1
2
5
14
42
132
429
1430
4862
16796
58786
208012
742900
2674440
9694845
35357670
129644790
477638700


In [47]:
rules = [Terminal('U', 1), Union('B', 'U', 'V'), Product('V', 'B', 'B')]
rules_sorted = sortRulesByValuation(rules)
for r in rules_sorted:
    print r.Value

U
V
B
